# Wpowadzenie do deep learning w bibliotece Flux.jl

## Wstęp

- [Flux](http://fluxml.ai/) jest biblioteką Julii przeznaczoną do tworzenia modeli uczenia maszynowego.
- Jest w całości oparta na Julii, przez co trywialne jest jej modyfikowanie i dostosowywanie do swoich potrzeb. 
- Możliwe jest przy tym wykorzystanie wewnątrz modeli składni, funkcji i makr Julii.
- Przy czym tworzenie całkiem złożonych standardowych modeli jest intuicyjne i szybkie, zazwyczaj zajmują one jedynie kilka linijek.

## Przykład

Aby w  zrozumieć sposób pracy z Fluxem warto rozpatrzeć prosty przykład. Zajmiemy się przetwarzaniem języka naturalnego - zbudujemy model zdolny do generowania składnej wypowiedzi w języku polskim.

Wyjściowe założenie jest takie, że wytrenujemy sieć neuronową, która będzie estymowała prawdopodobieństwo wystąpienia danego znaku w ciągu na podstawie poprzedzających go znaków w sekwencji ([<b>Character-Level Language Model</b>](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)).

Zbiórem na którym będziemy pracowali jest <i>W poszukiwaniu straconego czasu</i> Marcela Prousta. 

[![](https://upload.wikimedia.org/wikipedia/commons/b/b8/Marcel_Proust_1895.jpg)](https://pl.wikipedia.org/wiki/Marcel_Proust)

>(...) matka widząc, że mi jest zimno, namówiła mnie, abym się napił wbrew zwyczajowi trochę herbaty. Odmówiłem zrazu; potem, nie wiem czemu, namyśliłem się. Posłała po owe krótkie i pulchne ciasteczka zwane magdalenkami, które wyglądają jak odlane w prążkowanej skorupie muszli. I niebawem (...) machinalnie podniosłem do ust łyżeczkę herbaty, w której rozmoczyłem kawałek magdalenki. Ale w tej samej chwili, kiedy łyk pomieszany z okruchami ciasta dotknął mego podniebienia, zadrżałem, czując, że się we mnie dzieje coś niezwykłego. Owładnęła mną rozkoszna słodycz (...). Sprawiła, że w jednej chwili koleje życia stały mi się obojętne, klęski jako błahe, krótkość złudna (...). Cofam się myślą do chwili, w której wypiłem pierwszą łyżeczkę herbaty (...). I nagle wspomnienie zjawiło mi się. Ten smak to była magdalenka cioci Leonii.(...)

Zanim jednak zaczniemy wprowadźmy odrobinę teorii stojącej za tym zagadnieniem:

### Rekurencyjne sieci neuronowe (Recurrent neural networks)

- Charakterystyczną cechą tego typu sieci jest to, że pozwalają one na istnienie wewnątrz grafu cykli skierowanych.
- Oznacza to, że informacja wewnątrz takiej sieci nie musi płynąć tylko w jednym kierunku - neurony leżące na tej samej warstwie także mogą przesyłać sobie wzajemnie dane:

[![](http://karpathy.github.io/assets/rnn/diags.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

Dzięki tej właściwości RNN doskonale nadają się do budowy interesującego nas modelu: 

[![](http://karpathy.github.io/assets/rnn/charseq.jpeg)](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

### Long short-term memory

Problemem na który można natrafić w przypadku korzystania z RNN jest pamięć takiej sieci. Gdy odległość pomiędzy aktualnym a poprzedzającymi go węzłami, które niosą za sobą kluczową informację jest niewielka, sieć jest w stanie efektywnie je wykorzystać:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-shorttermdepdencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Problem się pojawia gdy ta odległość jest duża - wtedy kluczowe informacje po prostu znikają w szumie:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-longtermdependencies.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Wtedy też, warto zastosować sieć LSTM, która ze względu na swoją architekturę jest w stanie odpowiednio filtrować informację i wykorzystawać je nawet wtedy, gdy ich źródło jest znacznie oddalone od aktualnego neuronu:

[![](http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

Przejdźmy teraz do implementowania modelu za pomocą Fluxa:

### Implementacja

In [1]:
using Flux
using Flux: onehot, argmax, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

Pierwszym krokiem jest oczywiście odpowiednie przygotowanie danych na których będziemy pracowali:

In [2]:
text = collect(readstring("w_poszukiwaniu.txt"));
alphabet = [unique(text)..., '_'];

Następnie kodujemy zmienne jakościowe - wykorzystać do tego możemy funkcję [<tt>onehot</tt>](http://fluxml.ai/Flux.jl/stable/data/onehot.html):

In [3]:
text = map(ch -> onehot(ch, alphabet), text);
stop = onehot('_', alphabet);

In [4]:
N = length(alphabet);
seqlen = 50;
nbatch = 50;

In [5]:
Xs = collect(partition(batchseq(chunk(text, nbatch), stop), seqlen));
Ys = collect(partition(batchseq(chunk(text[2:end], nbatch), stop), seqlen));

In [6]:
q = chunk("ala ma kota", 3)

3-element Array{Array{Char,1},1}:
 ['a', 'l', 'a', ' ']
 ['m', 'a', ' ', 'k']
 ['o', 't', 'a']     

In [7]:
w = batchseq(q, '_')

4-element Array{Array{Char,1},1}:
 ['a', 'm', 'o']
 ['l', 'a', 't']
 ['a', ' ', 'a']
 [' ', 'k', '_']

In [8]:
for i in partition(w, 2)
   println(i)
end

Array{Char,1}[['a', 'm', 'o'], ['l', 'a', 't']]
Array{Char,1}[['a', ' ', 'a'], [' ', 'k', '_']]


In [9]:
collect(partition(w,2))

2-element Array{Array{Array{Char,1},1},1}:
 Array{Char,1}[['a', 'm', 'o'], ['l', 'a', 't']]
 Array{Char,1}[['a', ' ', 'a'], [' ', 'k', '_']]

In [10]:
collect(partition(batchseq(chunk("ala ma kota"[2:end], 3), '_'),2))

2-element Array{Array{Array{Char,1},1},1}:
 Array{Char,1}[['l', 'a', 't'], ['a', ' ', 'a']]
 Array{Char,1}[[' ', 'k', '_'], ['m', 'o', '_']]

####  Definiowanie modelu

Gdy dane są już gotowe kolejnym krokiem jest odpowiednie zdefiniowanie modelu na którym będziemy pracować. 

Zacznijmy od ręcznego zdefiniowania prostej regresji logistycznej (na początek załóżmy, że wagi są już zoptymalizowane):

In [11]:
W = rand(4, 8)
b = rand(4)

4-element Array{Float64,1}:
 0.575108
 0.910523
 0.464536
 0.920674

In [12]:
logit(x) = 1.0 ./ (1.0+exp.(-W*x .- b))

logit (generic function with 1 method)

In [13]:
x = rand(8)
logit(x)

4-element Array{Float64,1}:
 0.982528
 0.970391
 0.957775
 0.975082

Gdybyśmy chcieli wyuczyć ten model we Fluxie to powyższa definicja regresji logistycznej nam wystarczy - musimy jedynie zadeklerować <tt>W</tt> i <tt>b</tt> jako trenowalne parametry:

In [14]:
using Flux.Tracker

W = param(W)
b = param(b)

Tracked 4-element Array{Float64,1}:
 0.575108
 0.910523
 0.464536
 0.920674

Oczywiście pracując na Fluxie nie musimy deklarować wszystkiego ręcznie, dostarcza on najpopularniejsze [funkcje aktywacji](http://fluxml.ai/Flux.jl/stable/models/layers.html#Activation-Functions-1), które możemy wykorzystać w naszym modelu:

In [15]:
logit2(x) = σ.(W * x .+ b)

logit2 (generic function with 1 method)

In [16]:
logit2(x)

Tracked 4-element Array{Float64,1}:
 0.982528
 0.970391
 0.957775
 0.975082

Analogicznie nie ma konieczności definiowania [warstw modelu](http://fluxml.ai/Flux.jl/stable/models/layers.html#Basic-Layers-1) ręcznie:

In [17]:
logit3 = Dense(8,4,σ)

Dense(8, 4, NNlib.σ)

In [18]:
logit3(x)

Tracked 4-element Array{Float64,1}:
 0.466505
 0.364813
 0.566985
 0.21928 

Przy czym gdy żadna z dostarczonych razem z Fluxem definicji warstwy nam nie odpowiada możemy w banalny sposób zadeklarować własną:

In [19]:
struct Affine
  W
  b
end

Affine(in::Integer, out::Integer) =
  Affine(param(randn(out, in)), param(randn(out)))

# Overload call, so the object can be used as a function
(m::Affine)(x) = m.W * x.^2 .+ m.b

a = Affine(10, 5)

a(rand(10)) # => 5-element vector

Tracked 5-element Array{Float64,1}:
 -0.745883
 -0.205312
 -1.01167 
  2.52542 
  2.44472 

Gdy chcemy móc w pełni wykorzystać wszystkie przydatne funkcje Fluxa musimy jeszcze skorzystać z funkcji <tt>treelike</tt>:

In [20]:
Flux.treelike(Affine)

Wróćmy do generatora tekstu nad którym pracujemy. Zdefiniujemy go następująco:  

In [21]:
m = Chain(
  LSTM(N, 512),
  LSTM(512, 512),
  Dense(512, N),
  softmax)

Chain(Recur(LSTMCell(124, 512)), Recur(LSTMCell(512, 512)), Dense(512, 124), NNlib.softmax)

albo też:

In [22]:
layer1 = LSTM(N, 512);
layer2 = LSTM(512, 512);
layer3 = Dense(512, N);
layer4 = softmax;


In [23]:
m1(x) = layer4(layer3(layer2(layer1(x))))

m1 (generic function with 1 method)

lub:

In [24]:
m2(x) = layer1(x) |> layer2 |> layer3 |> layer4

m2 (generic function with 1 method)

czy też jako złożenie funkcji:

In [25]:
m3 = layer1 ∘ layer2 ∘ layer3 ∘ layer4 

(::#55) (generic function with 1 method)

Mając już gotową definicję modelu możemy przejść do kolejnego punktu - wyboru funkcji celu i regularyzacji modelu.

#### Funkcja straty, regularyzacja

Tak jak poprzednio funkcję straty możemy zdefiniować samodzielnie:

In [26]:
model = Dense(5,2)

x, y = rand(5), rand(2);

In [27]:
loss(ŷ, y) = sum((ŷ.- y).^2)/ length(y)

loss (generic function with 1 method)

In [28]:
loss(model(x), y) 

0.6380764053807677 (tracked)

Wykorzystać [jedną z zaimplementowanych we Fluxie:](https://github.com/FluxML/Flux.jl/blob/8f73dc6e148eedd11463571a0a8215fd87e7e05b/src/layers/stateless.jl)

In [29]:
Flux.mse(model(x),y)

0.6380764053807677 (tracked)

Albo wziąć ją z innej biblioteki:

In [30]:
using Distances
hellinger(model(x), y)

LoadError: DomainError:

Podobnie [regularyzacja](http://fluxml.ai/Flux.jl/stable/models/regularisation.html) jest dość [intuicyjna](http://fluxml.ai/Flux.jl/stable/models/layers.html#Normalisation-and-Regularisation-1):

In [31]:
penalty() = vecnorm(model.W) + vecnorm(model.b)
loss(ŷ,y) = Flux.mse(ŷ,y) + penalty()

loss (generic function with 1 method)

In [32]:
loss(model(x),y)

2.063592957484005 (tracked)

W budowanym modelu funkcja straty wyglądać będzie następująco:

In [33]:
function loss(xs, ys)
  l = sum(Flux.crossentropy.(m.(xs), ys))
  Flux.truncate!(m)
  return l
end

loss (generic function with 1 method)

#### Uczenie modelu

Po zdefiniowaniu modelu i funkcji celu możemy przystąpić do trenowania sieci.

Oczywiście podstawą, której potrzebujemy jest odpowiedni algorytm optymalizacyjny:

In [ ]:
function update!(ps, η = .0001)
  for w in ps
    w.data .-= w.grad .* η
    w.grad .= 0
  end
end



In [ ]:
i = 1
while true
  back!(loss(model(x),y))
  max(maximum(abs.(W.grad)), abs(b.grad[1])) > 0.001 || break
  update!((W, b))
  i += 1
end

Albo też wykorzystać [gotowy algorytm](http://fluxml.ai/Flux.jl/stable/training/optimisers.html) zaimplementowany we Fluxie: 

In [ ]:
opt = ADAM(params(m), 0.001)

Flux jest zdolny do kontrolowania całej procedury uczenia, służy do tego funkcja <tt>train!</tt>:

In [ ]:
Flux.train!(objective, data, opt)

Pozwala ona też na definiowanie wywołań, które pozwolą nam kontrolować przebieg uczenia.

Po omówieniu wszystkich elementów składowych biblioteki możemy złożyć naszą sieć w całość:

In [ ]:
m = Chain(
  LSTM(N, 512),
  LSTM(512, 512),
  Dense(512, N),
  softmax)

function loss(xs, ys)
  l = sum(crossentropy.(m.(xs), ys))
  Flux.truncate!(m)
  return l
end

opt = ADAM(params(m), 0.001)

#evalcb = () -> @show loss(Xs[5], Ys[5])

function sample(m, alphabet, len; temp = 1)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m(onehot(c, alphabet)).data)
  end
  return String(take!(buf))
end

evalcb = function ()
    @show loss(Xs[5], Ys[5])
    println(sample(deepcopy(m), alphabet, 100))
end

In [ ]:
@time Flux.train!(loss, zip(Xs, Ys), opt,
cb = throttle(evalcb, 240))

Oczywiście taka sieć może liczyć się strasznie długo. Ten proces można przyśpieszyć za pomocą wbudowanym w Julię wsparciu [obliczeń na GPU](http://fluxml.ai/Flux.jl/stable/gpu.html).

## Alternatywy

Flux nie jest jedyną biblioteką, która umożliwia budowanie modeli uczenia maszynowego w Julii. Poniżej wymienionych jest kilka alternatywnych możliwości:

- [Knet.jl](https://github.com/denizyuret/Knet.jl)
- [MXnet.jl](https://github.com/dmlc/MXNet.jl)
- [TensorFlow.jl](https://github.com/malmaud/TensorFlow.jl)